# Problem set 6: Solving the Solow model

In [2]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# Tasks

## Solving matrix equations I

In [3]:
np.random.seed(1900)
n = 5
A = np.random.uniform(size=(n,n))
b = np.random.uniform(size=n)
c = np.random.uniform(size=n)
d = np.random.uniform(size=n)

**Question A:** Find the determinant of $[A \cdot A]^{-1}$

In [17]:
# write your code here

# X = linalg.det(linalg.inv(A@A))

A_A = np.matmul(A, A) # A@A
A_A_inv = linalg.inv(A_A)
det = linalg.det(A_A_inv)

print(round(det, 2))

13132.55


**Answer:** see A1.py

**Question B:** Solve the following equation systems directly using **scipy**.

$$
\begin{aligned}
Ax &= b \\
Ax &= c \\
Ax &= d 
\end{aligned}
$$

In [27]:
xb = linalg.solve(A, b)
xc = linalg.solve(A, c)
xd = linalg.solve(A, d)

print('b =', np.round(xb, 2))
print('c =', np.round(xc, 2))
print('d =', np.round(xd, 2))

b = [-15.33 -24.01  40.03  15.24   4.89]
c = [ -7.72 -11.64  20.87   7.93   1.56]
d = [-2.57 -4.    7.93  2.56  1.26]


**Answer:** A2.py

**Question C:** Solve the same equation systems as above using `linalg.lu_factor()` and `linalg.lu_solve()`. What is the benefit of this approach?

In [36]:
lu, piv = linalg.lu_factor(A) # done only once

sol_b = linalg.lu_solve((lu, piv), b) # faster than regular linalg.solve
sol_c = linalg.lu_solve((lu, piv), c)
sol_d = linalg.lu_solve((lu, piv), d)

print('b =', np.round(sol_b, 2))
print('c =', np.round(sol_c, 2))
print('d =', np.round(sol_d, 2))

b = [-15.33 -24.01  40.03  15.24   4.89]
c = [ -7.72 -11.64  20.87   7.93   1.56]
d = [-2.57 -4.    7.93  2.56  1.26]


**Answer:** A3.py

## Solving matrix equations II

In [37]:
F = np.array([[2.0, 1.0, -1.0], [-3.0, -1.0, 2], [-2.0, 1.0, 2.0]])
e = np.array([8.0, -11.0, -3.0])

**Question:** Use the function `gauss_jordan()` in the `numecon_linalg` module located in this folder to solve

$$
Fx = e
$$

In [60]:
# write your code here

import numecon_linalg as nl


# column_stack: take a sequence of 1-D arrays and stack them as columns to make a single 2-D array
X = np.column_stack((F, e)) # X = np.vstack((F, e)).T

# Gauss-Jordan elimination on the matrix X
nl.gauss_jordan(X)

# prints only last column of X, assuming the solution is stored in that column
print('solution', X[:,-1])

assert np.allclose(F@X[:,-1], e)


solution [ 2.  3. -1.]


**Answer:** see A4.py

## Symbolic

**Question A:** Find

$$
\lim_{x \rightarrow 0} \frac{\sin(x)}{x}
$$

and

$$
\frac{\partial\sin(2x)}{\partial x} 
$$

In [78]:
# write your code here - limit

x = sm.symbols('x')
expr = sin(x)/x

limit_expr = sm.limit(expr, x, 0)

print('Limit of the expression tends to 0 at:', limit_expr)

Limit of the expression tends to 0 at: 1


In [84]:
# write your code here - derivative

x = sm.symbols('x')
expr = sin(2*x)

sm.diff(expr)

# print('The derivative of the expression is:', dev)

2*cos(2*x)

**Answer:** A5.py

**Question B:** Solve the equation

$$ 
\frac{\sin(x)}{x} = 0
$$

In [86]:
# write your code here

expr = sin(x)/x 

sm.solve(expr, 0)

AttributeError: module 'numpy' has no attribute 'solve'

**Answer:** A6.py

# Problem: Solve the Solow model

## Introduction

Consider the **standard Solow-model** where:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$

## Steady state

Assume the production function is **Cobb-Douglas**.

**Question A:** Use **sympy** to find an analytical expression for the steady state, i.e. solve

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

In [15]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

In [16]:
# write your code here

**Answer:** see A7.py

**Question B:** Turn you solution into a Python function called as `ss_func(s,g,n,delta,alpha)`. 

In [18]:
# write your code here

**Answer:** A8.py

**Question C**: Find the steady state numerically using root-finding with `optimize.root_scalar`.

In [21]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

# write your code here

**Answer:** A9.py

**Question D:** Now assume the production function is CES. Find the steady state for $k$ for the various values of $\beta$ shown below.

In [23]:
betas = [-0.5,-0.25,-0.1,-0.05,0.05,0.1,0.25,0.5]

# write your code here

**Answer:** A10.py   